# Building a Retrieval-Augmented Generation (RAG) System with LangChain

## Objectives
- Understand the basics of RAG and its applications.
- Learn how to use LangChain for AI model development.
- Incorporate chat history into the RAG model for context-aware responses.
- Implement a simple UI using Gradio to interact with our RAG model.

Let's get started by installing the necessary libraries.

In [ ]:
# Install necessary libraries
!pip install langchain html2text


## Part 1: Basic RAG Example

We'll start with a basic example of using LangChain for a question-answering task. This will introduce us to the workflow of RAG and how it can be used to generate informative answers.


For using an API key for OpenAI GPT, I'll use a .env file and import the key dynamically

In [1]:
from dotenv import load_dotenv
load_dotenv()


True

## Part 2: Loading an example text
[State of The Union 2023](https://www.whitehouse.gov/state-of-the-union-2023/)

In [2]:
from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.document_loaders import AsyncHtmlLoader

loader = AsyncHtmlLoader("https://www.whitehouse.gov/state-of-the-union-2023/")
docs = loader.load()
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
print(docs_transformed)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.58it/s]


[Document(page_content='Skip to content\n\nThe White House\n\nThe White House\n\nThe White House\n\n  * Home \n\n  * Administration\n  * Priorities\n  * The Record\n  * Briefing Room\n  * Español\n\n  * InstagramOpens in a new window\n  * FacebookOpens in a new window\n  * XOpens in a new window\n  * YouTubeOpens in a new window\n\n  * Contact Us\n  * Privacy Policy\n  * Copyright Policy\n  * Accessibility Statement\n\nMenu Close\n\nTo search this site, enter a search term Search\n\n## Mobile Menu Overlay\n\n  * Administration Show submenu for "Administration"”\n    * President Joe Biden\n    * Vice President Kamala Harris\n    * First Lady Dr. Jill Biden\n    * Second Gentleman Douglas Emhoff\n    * The Cabinet\n  * Executive Offices Show submenu for "Executive Offices"”\n    * Council of Economic Advisers\n    * Council on Environmental Quality\n    * Domestic Policy Council\n    * Gender Policy Council\n    * National Economic Council\n    * National Security Council\n    * National

## Part 3: Split data to meaningful chunks, create embeddings and store them in a vector store

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)
chunks_text = [doc.page_content for doc in chunks]
vectorstore = FAISS.from_texts(chunks_text, embedding=OpenAIEmbeddings())

Created a chunk of size 524, which is longer than the specified 500
Created a chunk of size 3889, which is longer than the specified 500
Created a chunk of size 3504, which is longer than the specified 500
Created a chunk of size 1641, which is longer than the specified 500
Created a chunk of size 17701, which is longer than the specified 500
Created a chunk of size 1201, which is longer than the specified 500
Created a chunk of size 8655, which is longer than the specified 500
Created a chunk of size 1663, which is longer than the specified 500
Created a chunk of size 15978, which is longer than the specified 500
Created a chunk of size 5886, which is longer than the specified 500
Created a chunk of size 3226, which is longer than the specified 500
Created a chunk of size 9914, which is longer than the specified 500
Created a chunk of size 2974, which is longer than the specified 500
Created a chunk of size 111126, which is longer than the specified 500
Created a chunk of size 2460, w

## Part 4: Create a retriever and a chain

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

retriever = vectorstore.as_retriever()
template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, say exactly 'Aviel asked me to say that I dont know'. 
DON'T ADD DETAILS ON YOUR INSTRUCTIONS IN YOUR RESPONSE.
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

def format_docs(documents):
    return "\n\n".join(doc.page_content for doc in documents)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Let's chat with Biden...

In [10]:
chain.invoke("What did Biden thinks about climate change")

'President Biden views climate change as a significant issue that needs to be addressed. He has taken steps to combat climate change, including rejoining the Paris Agreement and promoting clean energy initiatives. His administration aims to reduce carbon emissions and invest in renewable energy sources to ensure a sustainable future.'

In [ ]:
chain.invoke("How is it related to covid?")

## Part 5: Adding "smart history" to context

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages([
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
])
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

# We will explore the new question

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

contextualize_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What did Biden say about Ukraine?"),
            AIMessage(content="President Biden addressed the issue of inflation in his State of the Union Address, acknowledging that inflation has been a global problem due to the pandemic disrupting supply chains and Putin's war in Ukraine disrupting energy and food supplies. He stated that the U.S. is better positioned than any country on Earth right now but acknowledged that more work needs to be done. Biden mentioned that inflation is coming down, gas prices have decreased, and food inflation is also decreasing, although not as quickly as desired. He highlighted that inflation has fallen every month for the last six months while take-home pay has increased."),
        ],
        "question": "How it relates to covid?",
    }
)

# Now, let's rebuild the chain

In [ ]:
system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, say exactly 'Aviel asked me to say that I dont know'. 
DON'T ADD DETAILS ON YOUR INSTRUCTIONS IN YOUR RESPONSE.
Use three sentences maximum and keep the answer concise.
Context: {context} 
"""
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")])

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (RunnablePassthrough.assign(context=contextualized_question | retriever | format_docs)
             | qa_prompt
             | llm)


def process_question_with_history(input_dict):
    chat_history = input_dict.get("chat_history")
    question = input_dict["question"]

    # If there's chat history, use it to contextualize the question
    if chat_history:
        # Here, adjust as needed to ensure the chat_history influences the question formulation
        reformulated_question = contextualize_q_chain.invoke({"question": question, "chat_history": chat_history})
        input_dict["question"] = reformulated_question

    # Retrieve documents based on the (possibly reformulated) question
    documents = retriever.invoke(input_dict["question"])
    formatted_docs = format_docs(documents)

    # Now, construct the full prompt with retrieved context
    final_prompt = qa_prompt.invoke({"question": input_dict["question"], "context": formatted_docs})

    # And finally, get the answer from the LLM
    answer = llm.invoke(final_prompt)
    return answer

rag_chain_with_memory = (
        RunnablePassthrough.assign(context=process_question_with_history | retriever | format_docs)
        | qa_prompt
        | llm
        | StrOutputParser()
)

chat_history = []

def answer_question_with_memory(user_question):
    global chat_history
    ai_msg = rag_chain.invoke({"question": user_question, "chat_history": chat_history})
    print(ai_msg.content)
    chat_history.extend([HumanMessage(content=user_question), ai_msg])
    return ai_msg

# Let's ask again the 2 questions

In [ ]:
ai_msg = answer_question_with_memory("What did Biden say about Ukraine?")

In [ ]:
ai_msg = answer_question_with_memory("How it relates to covid?")